In [ ]:
"""Reproduce Fig3 of [2D Mater. 9, 015008 (2022)]."""


In [ ]:
import numpy as np
from scipy.constants import pi
import scipy.linalg as la
import matplotlib
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

# Set the figure font
matplotlib.rcParams["font.family"] = "STIXGeneral"
matplotlib.rcParams["font.serif"] = "STIXGeneral"
matplotlib.rcParams["mathtext.fontset"] = "stix"

In [ ]:
sigma_0 = np.array([[1, 0], [0, 1]])
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

s_0 = np.array([[1, 0], [0, 1]])
s_x = np.array([[0, 1], [1, 0]])
s_y = np.array([[0, -1j], [1j, 0]])
s_z = np.array([[1, 0], [0, -1]])

In [ ]:
def H(kx, ky, **kwargs):
    """Define the kp Hamiltonian of graphene."""

    # This kp Hamiltonian takes Dirac point as the original point, namely, K = [0, 0].
    # Thus there is no need to subtract K point from kpoint.

    hbar = kwargs["hbar"]
    vF = kwargs["vF"]
    Delta = kwargs["Delta"]
    lambda_I = kwargs["lambda_I"]
    lambda_VZ = kwargs["lambda_VZ"]
    lambda_R = kwargs["lambda_R"]
    tau = kwargs["tau"]
    tau_z = kwargs["tau"]  # I doubt that both tau and tau_z denote K or K' by 1 or -1

    # I find the H_R term of this paper is strange, since it givens no info about Kappa.
    # I suppose the H_R term should be consistent with PRL2022.
    # If so, Kappa can only be -1 to keep consistent with PRL2022.
    kappa = -1

    # basis:
    # c1_up, c1_dn, c2_up, c2_dn
    # Thus, spin operator at the latter
    H_0 = hbar * vF * np.kron((tau * kx * sigma_x + ky * sigma_y), s_0)
    H_D = Delta * np.kron(sigma_z, s_0)
    H_I = lambda_I * tau_z * np.kron(sigma_z, s_z)
    H_VZ = lambda_VZ * tau_z * np.kron(sigma_0, s_z)
    H_R = lambda_R * (kappa * np.kron(sigma_x, s_y) - np.kron(sigma_y, s_x))
    # (...) * -1 is to reproduce Fig3a, where VB difference is larger at Gamma.
    H = H_0 + H_D + H_I * -1 + H_VZ + H_R

    eigenvalues, eigenvectors = la.eigh(H)

    # Arrange the eigenvalues from the smallest to the largest by argsort()[::-1],
    # while keep the correct corresponding ordering of eigenvectors
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    return eigenvalues, eigenvectors

In [ ]:
def rotate_vector(vector, angle):
    """Illustrate the rotation of b vectors in BZ."""
    rotation_matrix = np.array(
        [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
    )
    return rotation_matrix @ vector


# Define the initial vectors
angle1 = 0  # First vector at 0 degrees
angle2 = np.radians(120)  # Second vector at 120 degrees
magnitude = 1  # Length of the vectors

# Create the initial vectors
vector1 = np.array([magnitude * np.cos(angle1), magnitude * np.sin(angle1)])
vector2 = np.array([magnitude * np.cos(angle2), magnitude * np.sin(angle2)])

# Rotate the vectors by a specified angle
rotation_angle = np.radians(-15)
rotated_vector1 = rotate_vector(vector1, rotation_angle)
rotated_vector2 = rotate_vector(vector2, rotation_angle)

# Generate points along the new vectors
num_points = 10
step_size = 0.5  # Distance between points along each vector

points_along_rotated_vector1 = np.array(
    [n * step_size * rotated_vector1 for n in range(num_points)]
)
points_along_rotated_vector2 = np.array(
    [n * step_size * rotated_vector2 for n in range(num_points)]
)

# Plot the results
plt.figure(figsize=(8, 8))

# Plot original vectors
plt.quiver(
    0,
    0,
    vector1[0],
    vector1[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="blue",
    label="Original Vector 1",
)
plt.quiver(
    0,
    0,
    vector2[0],
    vector2[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="green",
    label="Original Vector 2",
)

# Plot rotated vectors
plt.quiver(
    0,
    0,
    rotated_vector1[0],
    rotated_vector1[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="red",
    label="Rotated Vector 1",
)
plt.quiver(
    0,
    0,
    rotated_vector2[0],
    rotated_vector2[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="purple",
    label="Rotated Vector 2",
)

# Plot points along the rotated vectors
plt.plot(
    points_along_rotated_vector1[:, 0],
    points_along_rotated_vector1[:, 1],
    "ro",
    label="Points along Rotated Vector 1",
)
plt.plot(
    points_along_rotated_vector2[:, 0],
    points_along_rotated_vector2[:, 1],
    "bo",
    label="Points along Rotated Vector 2",
)

# Formatting the plot
plt.axhline(0, color="black", linewidth=0.5)
plt.axvline(0, color="black", linewidth=0.5)
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.gca().set_aspect("equal", adjustable="box")
plt.legend()
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("Original and Rotated Vectors with Points Along New Directions")
plt.grid()
plt.show()

In [ ]:
def generate_points_120deg(**kwargs):
    """Generate a list of 2D points along two directions with 120 deg angle.

    Return:
        array: An array including kpoints along two directions.
    """
    # A problem of this paper is that Fig3 doesn't clearly present the kpath direction.
    # Thus I can only guess, then check my assumption by whether I can reproduce.
    # The first intuitive guess was MKG, but failed.
    # This guess where kpath forming an 120 deg angle is confirmed by reproduction.
    # Note that due to the proximity effect, two vectors of kpath are not equivalent.

    # Define the rotation function. angle is the rotating angle.
    def rotate_vector(vector, angle):
        rotation_matrix = np.array(
            [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
        )
        return rotation_matrix @ vector

    angle1 = 0  # First vector at 0 degrees
    angle2 = np.radians(120)  # Second vector at 120 degrees

    # Create the initial vectors
    vector1 = np.array([np.cos(angle1), np.sin(angle1)])
    vector2 = np.array([np.cos(angle2), np.sin(angle2)])

    # Rotate the vectors by a specified angle
    rotation_angle = np.radians(kwargs["angle"])  # Rotation angle
    rotated_vector1 = rotate_vector(vector1, rotation_angle)
    rotated_vector2 = rotate_vector(vector2, rotation_angle)

    # Generate points along the new vectors
    num_points = int(kwargs["num_points"] / 2)
    # Distance between points along each vector
    step_size = (kwargs["xmax"] - kwargs["xmin"]) / kwargs["num_points"]

    points_along_rotated_vector1 = np.array(
        [n * step_size * rotated_vector1 for n in range(num_points)]
    )
    points_along_rotated_vector2 = np.array(
        [n * step_size * rotated_vector2 for n in range(num_points)]
    )

    all_points = np.vstack(
        (points_along_rotated_vector1[::-1], points_along_rotated_vector2)
    )

    return all_points

In [ ]:
def calc_bands(**kwargs):
    """Calculate eigenvalues of given kpoints.

    Returns:
        kpath (array)
        energies (array): Eigenvalues of all kpoints and bands.
    """
    points = generate_points_120deg(**kwargs)
    energies = np.zeros((kwargs["num_points"], kwargs["nbnd"]))
    kpath = np.zeros(kwargs["num_points"])
    for i in range(kwargs["num_points"]):
        kx = points[i][0]
        ky = points[i][1]
        # Generate kpath which is centered at [0, 0] point
        # Since the kpath has a 120 deg angle,
        # we can cumulate the kpath since any direction.
        # The given kpath is to reproduce Fig3.
        kpath[i] = np.sqrt(kx**2 + ky**2) * -1 if kx > 0 else np.sqrt(kx**2 + ky**2)
        # Calculate eigenvalues
        e, k = H(kx, ky, **kwargs)
        energies[i, :] = e

    return kpath, energies

In [ ]:
def calc_spins(operator, **kwargs):
    """Calculate spin expectations of given kpoints.

    Parameters:
        operator (array): Pauli matrix.
    Returns:
        kpath (array)
        spins (array): Eigenvalues of all kpoints and bands.
    """
    points = generate_points_120deg(**kwargs)
    spins = np.zeros((kwargs["num_points"], kwargs["nbnd"]))
    kpath = np.zeros(kwargs["num_points"])
    for i in range(kwargs["num_points"]):
        kx = points[i][0]
        ky = points[i][1]
        # Generate kpath which is centered at [0, 0] point
        # Since the kpath has a 120 deg angle,
        # we can cumulate the kpath since any direction.
        # The given kpath is to reproduce Fig3.
        kpath[i] = np.sqrt(kx**2 + ky**2) * -1 if kx > 0 else np.sqrt(kx**2 + ky**2)

        # Generate spin array with shape of (num_points, nbnd)
        e, k = H(kx, ky, **kwargs)  # Get eigenvals and eigenfuncs
        spin_k = np.zeros(kwargs["nbnd"])
        for j in range(kwargs["nbnd"]):
            spin = (
                np.transpose(np.conjugate(k[:, j].reshape(kwargs["nbnd"], 1)))
                @ np.kron(np.eye(2), operator)
                @ k[:, j].reshape(kwargs["nbnd"], 1)
            )  # Pauli mat is 2x2, operating on the vector
            spin_k[j] = spin.item().real
        spins[i, :] = spin_k  # Define max as 0.5, thus the spin unit is hbar/2

    # kpath = kpath[::-1]
    return kpath, spins

In [ ]:
def plot_spinz_bands(**kwargs):
    """Plot bands projected by spinz."""

    kpath, energies = calc_bands(**kwargs)
    kpath, spins = calc_spins(operator=s_z, **kwargs)

    for i in range(kwargs["nbnd"]):
        plt.scatter(kpath, energies[:, i], c=spins[:, i], s=1, cmap="coolwarm")
        plt.clim(-0.5, 0.5)  # max spin is 0.5 acc to left figures in Fig6

    plt.tick_params(axis="x", which="both", top=False, direction="in")
    plt.tick_params(axis="y", which="both", top=False, direction="in")
    plt.hlines(0, plt.xlim()[0], plt.xlim()[1], lw=0.5, colors="gray")
    plt.ylabel(r"$\mathrm{E}-\mathrm{E}_\mathrm{F}$ [meV]")
    # plt.ylim(-12, 8)
    plot_ticks(**kwargs)

In [ ]:
def plot_spins(operator, **kwargs):
    """Plot spin expectation values."""

    kpath, spins = calc_spins(operator, **kwargs)
    for i in range(kwargs["nbnd"]):
        # Ordering from low-energy band as given in H(kx,ky).
        if i % 2 == 0:
            plt.plot(kpath, spins[:, i], c="b")
        else:
            plt.plot(kpath, spins[:, i], c="r")
    plt.ylim(-1, 1)
    plot_ticks(**kwargs)

In [ ]:
def plot_ticks(**kwargs):
    """Format figure."""

    # ax = plt.gca()
    plt.xlim(kwargs["xmin"], kwargs["xmax"])
    plt.tick_params(axis="both", which="both",   direction="in")
    plt.xlabel(r"k - K [nm$^{-1}$]") 

    plt.tight_layout()

In [ ]:
def plot_figure(**kwargs):
    """Plot figure as Fig3."""

    fig = plt.figure(figsize=(5, 3.5), dpi=144)
    # Control the scale of rows and columns in figure
    a = 1
    c = 0.5
    gs = GridSpec(
        3,
        3,
        width_ratios=(a, c, a),
        height_ratios=(a, c, a),
        hspace=0,
        wspace=0,
        figure=fig,
    )

    plt.subplot(gs[0, 0])
    plot_spinz_bands(**kwargs)

    plt.subplot(gs[0, 2])
    plot_spins(operator=s_x, **kwargs)
    plt.ylabel("$s_x$ ($\hbar/2$)")

    plt.subplot(gs[2, 0])
    plot_spins(operator=s_y, **kwargs)
    plt.ylabel("$s_y$ ($\hbar/2$)")

    plt.subplot(gs[2, 2])
    plot_spins(operator=s_z, **kwargs)
    plt.ylabel("$s_z$ ($\hbar/2$)")

In [ ]:
# Define physical parameters.
# The length unit is nm. Remember to change constant values if you change the unit.
# The values are read from Fig4b, the red bar.
kwargs = dict(
    vF=8.5e5 * 1e9,  # velocity in nm/s. Value not given. Estimate acc to band dispersion.
    hbar=6.582119569e-16 * 1e3,  # hbar in meV⋅s
    Delta=0.038,  # in meV
    lambda_R=0.07,  # in meV
    lambda_I=0.033,  # in meV
    lambda_VZ=0.052,  # in meV
    tau=1,
    angle=-15,  # in degree
    xmin=-2.5e-3,  # in nm-1
    xmax=2.5e-3,  # in nm-1
    num_points=100,  # dimensionless
    nbnd=4,  # dimensionless
)


def main():
    plot_figure(**kwargs)
    plt.show()

In [ ]:
if __name__ == "__main__":
    main()